In [ ]:
### This notebook use the Open Route Service to download edge features (including highway segment geometries)
### Deprecated due to low quality, the paper use  the Google Route API instead

In [1]:
import pandas as pd
import requests
from tqdm import tqdm  # For progress bar
import time
import os
import json

# Ensure the folder exists
os.makedirs("edge_features_json_ors", exist_ok=True)

# Load the input files
nodes_df = pd.read_csv("tris_node_features-v1.1.csv")
sensors_df = pd.read_csv("sensors_498.csv")

# API setup
API_KEY = "5b3ce3597851110001cf62484f16cdedfd154831853bdc704490cd97"
API_URL_LOCAL = "http://localhost:8080/ors/v2/directions/driving-car/json"
API_URL = "https://api.openrouteservice.org/v2/directions/driving-car/json"
HEADERS = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': API_KEY,
    'Content-Type': 'application/json; charset=utf-8'
}

# Function to call the OpenRouteService API
def fetch_route_data(origin, destination):
    body = {
        "coordinates": [origin, destination],
        "attributes": ["avgspeed", "detourfactor", "percentage"],
        "extra_info": ["waytype"],
        "units": "m"
    }
    response = requests.post(API_URL, json=body, headers=HEADERS)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response}")
        return None


In [3]:
# Processing each sensor
edge_features = []

# Use tqdm for progress bar
for _, sensor in tqdm(sensors_df.iterrows(), total=sensors_df.shape[0], desc="Processing Sensors"):
    json_filename = f"edge_features_json_ors/sensor_{int(sensor['Id'])}.json"
    
    if os.path.exists(json_filename):
        # Load JSON file if it exists
        with open(json_filename, "r") as json_file:
            route_data = json.load(json_file)
    else:
        # Get node coordinates
        origin_node = nodes_df.loc[nodes_df['Node_Id'] == sensor['Origin']]
        destination_node = nodes_df.loc[nodes_df['Node_Id'] == sensor['Destination']]

        if origin_node.empty or destination_node.empty:
            print(f"Missing node data for sensor {sensor['Id']}")
            continue

        origin_coords = [origin_node.iloc[0]['Longitude'], origin_node.iloc[0]['Latitude']]
        destination_coords = [destination_node.iloc[0]['Longitude'], destination_node.iloc[0]['Latitude']]

        # Fetch route data from API
        route_data = fetch_route_data(origin_coords, destination_coords)
        if not route_data:
            edge_features.append({
                "Sensor_Id": None,
                "Duration": None,
                "Distance": None,
                "WayType_1": None,
                "WayType_2": None,
                "WayType_Other": None,
                "AvgSpeed": None,
                "DetourFactor": None
            })
            continue

        # Save the JSON response to a file
        with open(json_filename, "w") as json_file:
            json.dump(route_data, json_file, indent=4)

    # Extract features
    duration = route_data["routes"][0]["summary"]["duration"]
    distance = route_data["routes"][0]["summary"]["distance"]
    
    # Extract waytype summary
    waytype_summary = route_data["routes"][0]["extras"]["waytypes"]["summary"]
    waytype_embedding = [0, 0, 0]  # Only considering types 1 and 2; rest default to 0
    
    for summary in waytype_summary:
        waytype = summary["value"]
        amount = summary["amount"] / 100  # Convert percentage to fraction
        if waytype == 1:
            waytype_embedding[0] += amount
        elif waytype == 2:
            waytype_embedding[1] += amount
        else:
            waytype_embedding[2] += amount
    
    # Extract avg_speed and detour_factor
    avg_speed = sum(
        seg["avgspeed"] * seg["percentage"] / 100
        for seg in route_data["routes"][0]["segments"]
    )
    detour_factor = sum(
        seg["detourfactor"] * seg["percentage"] / 100
        for seg in route_data["routes"][0]["segments"]
    )
    
    # Append features to list
    edge_features.append({
        "Sensor_Id": int(sensor["Id"]),
        "Duration": duration,
        "Distance": distance,
        "WayType_1": waytype_embedding[0],
        "WayType_2": waytype_embedding[1],
        "WayType_Other": waytype_embedding[2],
        "AvgSpeed": avg_speed,
        "DetourFactor": detour_factor
    })
    time.sleep(0)

# Save the features to a CSV file
edge_features_df = pd.DataFrame(edge_features)
edge_features_df.to_csv("tris_edge_features-v1.1-api.csv", index=False)

Processing Sensors: 100%|██████████| 498/498 [00:00<00:00, 4936.57it/s]


In [67]:
len(edge_features)

498